In [100]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
import pandas as pd
import numpy as np

In [2]:
YEARS = [2019,2018,2017]

In [3]:
data = pd.DataFrame()

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    data = data.append(i_data, sort=True)

In [5]:
data.reset_index(drop=True, inplace=True)

In [6]:
data.head()

,aborted_play,air_epa,air_wpa,air_yards,assist_tackle,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,...,wp,wpa,yac_epa,yac_wpa,yardline_100,yards_after_catch,yards_gained,ydsnet,ydstogo,yrdln
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.445672,0.000000,NaN,NaN,35.0,NaN,0.0,8.0,0,MIN 35
2,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.445672,-0.064775,NaN,NaN,75.0,NaN,-8.0,8.0,10,ATL 25
3,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.380897,-0.009275,NaN,NaN,83.0,NaN,4.0,8.0,18,ATL 17
4,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.371623,0.015402,NaN,NaN,79.0,NaN,12.0,8.0,14,ATL 21


In [8]:
for col in data.columns: 
    print(col)

aborted_play
air_epa
air_wpa
air_yards
assist_tackle
assist_tackle_1_player_id
assist_tackle_1_player_name
assist_tackle_1_team
assist_tackle_2_player_id
assist_tackle_2_player_name
assist_tackle_2_team
assist_tackle_3_player_id
assist_tackle_3_player_name
assist_tackle_3_team
assist_tackle_4_player_id
assist_tackle_4_player_name
assist_tackle_4_team
away_coach
away_score
away_team
away_timeouts_remaining
away_wp
away_wp_post
blocked_player_id
blocked_player_name
comp_air_epa
comp_air_wpa
comp_yac_epa
comp_yac_wpa
complete_pass
cp
cpoe
def_wp
defensive_extra_point_attempt
defensive_extra_point_conv
defensive_two_point_attempt
defensive_two_point_conv
defteam
defteam_score
defteam_score_post
defteam_timeouts_remaining
desc
div_game
down
drive
drive_end_transition
drive_end_yard_line
drive_ended_with_score
drive_first_downs
drive_game_clock_end
drive_game_clock_start
drive_inside20
drive_play_count
drive_play_id_ended
drive_play_id_started
drive_quarter_end
drive_quarter_start
drive_real

In [35]:
data.posteam_type.unique()

array([nan, 'away', 'home'], dtype=object)

In [36]:
df = data[['game_date', 'home_team', 'away_team', 'posteam', 'defteam', 'posteam_type', 'yardline_100','quarter_seconds_remaining','half_seconds_remaining','game_seconds_remaining','qtr','down','goal_to_go','ydstogo','play_type','score_differential','shotgun','no_huddle']]
df = df.dropna()

In [37]:
df.isnull().sum()

game_date                    0
home_team                    0
away_team                    0
posteam                      0
defteam                      0
posteam_type                 0
yardline_100                 0
quarter_seconds_remaining    0
half_seconds_remaining       0
game_seconds_remaining       0
qtr                          0
down                         0
goal_to_go                   0
ydstogo                      0
play_type                    0
score_differential           0
shotgun                      0
no_huddle                    0
dtype: int64

In [38]:
df.play_type.unique()

array(['pass', 'run', 'punt', 'no_play', 'qb_kneel', 'field_goal',
       'qb_spike'], dtype=object)

In [39]:
dftest = df[df['play_type'].isin(['run', 'pass'])]

In [40]:
dftest

,game_date,home_team,away_team,posteam,defteam,posteam_type,yardline_100,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_differential,shotgun,no_huddle
2,2019-09-08,MIN,ATL,ATL,MIN,away,75.0,900.0,1800.0,3600.0,1,1.0,0,10,pass,0.0,0,0
3,2019-09-08,MIN,ATL,ATL,MIN,away,83.0,860.0,1760.0,3560.0,1,2.0,0,18,run,0.0,0,0
4,2019-09-08,MIN,ATL,ATL,MIN,away,79.0,821.0,1721.0,3521.0,1,3.0,0,14,run,0.0,1,0
7,2019-09-08,MIN,ATL,MIN,ATL,home,31.0,752.0,1652.0,3452.0,1,1.0,0,20,pass,0.0,0,0
8,2019-09-08,MIN,ATL,MIN,ATL,home,23.0,717.0,1617.0,3417.0,1,2.0,0,12,pass,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143897,2018-02-04,NE,PHI,NE,PHI,home,91.0,42.0,42.0,42.0,4,4.0,0,10,pass,-8.0,1,0
143899,2018-02-04,NE,PHI,NE,PHI,home,78.0,26.0,26.0,26.0,4,2.0,0,10,pass,-8.0,1,0
143900,2018-02-04,NE,PHI,NE,PHI,home,67.0,20.0,20.0,20.0,4,1.0,0,10,pass,-8.0,1,0
143902,2018-02-04,NE,PHI,NE,PHI,home,51.0,13.0,13.0,13.0,4,1.0,0,10,pass,-8.0,1,0


In [41]:
print(df['down'].unique())
print(df['qtr'].unique())
print(df['goal_to_go'].unique())
print(df['shotgun'].unique())
print(df['no_huddle'].unique())

[1. 2. 3. 4.]
[1 2 3 4 5]
[0 1]
[0 1]
[0 1]


In [134]:
eagles_df = df[(df.posteam == 'PHI')]

In [135]:
eagles_df = eagles_df[eagles_df['play_type'].isin(['run', 'pass'])]

In [136]:
eagles_df.reset_index(drop=True, inplace=True)

In [137]:
eagles_df.home_team.nunique()

20

In [138]:
eagles_df.game_date.unique()

array(['2019-09-08', '2019-09-15', '2019-09-22', '2019-09-26',
       '2019-10-06', '2019-10-13', '2019-10-20', '2019-10-27',
       '2019-11-03', '2019-11-17', '2019-11-24', '2019-12-01',
       '2019-12-09', '2019-12-15', '2019-12-22', '2019-12-29',
       '2020-01-05', '2018-09-06', '2018-09-16', '2018-09-23',
       '2018-09-30', '2018-10-07', '2018-10-11', '2018-10-21',
       '2018-10-28', '2018-11-11', '2018-11-18', '2018-11-25',
       '2018-12-03', '2018-12-09', '2018-12-16', '2018-12-23',
       '2018-12-30', '2019-01-06', '2019-01-13', '2017-09-10',
       '2017-09-17', '2017-09-24', '2017-10-01', '2017-10-08',
       '2017-10-12', '2017-10-23', '2017-10-29', '2017-11-05',
       '2017-11-19', '2017-11-26', '2017-12-03', '2017-12-10',
       '2017-12-17', '2017-12-25', '2017-12-31', '2018-01-13',
       '2018-01-21', '2018-02-04'], dtype=object)

In [139]:
eagles_df

,game_date,home_team,away_team,posteam,defteam,posteam_type,yardline_100,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_differential,shotgun,no_huddle
0,2019-09-08,PHI,WAS,PHI,WAS,home,75.0,654.0,1554.0,3354.0,1,1.0,0,10,pass,-7.0,1,0
1,2019-09-08,PHI,WAS,PHI,WAS,home,75.0,648.0,1548.0,3348.0,1,2.0,0,10,pass,-7.0,0,0
2,2019-09-08,PHI,WAS,PHI,WAS,home,64.0,617.0,1517.0,3317.0,1,1.0,0,10,run,-7.0,0,0
3,2019-09-08,PHI,WAS,PHI,WAS,home,62.0,575.0,1475.0,3275.0,1,2.0,0,8,run,-7.0,1,0
4,2019-09-08,PHI,WAS,PHI,WAS,home,71.0,539.0,1439.0,3239.0,1,3.0,0,17,pass,-7.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3526,2018-02-04,NE,PHI,PHI,NE,away,11.0,150.0,150.0,150.0,4,2.0,0,7,pass,-1.0,1,0
3527,2018-02-04,NE,PHI,PHI,NE,away,11.0,145.0,145.0,145.0,4,3.0,0,7,pass,-1.0,1,0
3528,2018-02-04,NE,PHI,PHI,NE,away,31.0,129.0,129.0,129.0,4,1.0,0,10,run,5.0,0,0
3529,2018-02-04,NE,PHI,PHI,NE,away,29.0,123.0,123.0,123.0,4,2.0,0,8,run,5.0,0,0


eagles_df["posteam_type"].replace({"home": "1", "away": "0"}, inplace=True)

eagles_df

eagles_df.dtypes

eagles_df['posteam_type'] = eagles_df['posteam_type'].astype(str).astype(int)

In [113]:
eagles_df.dtypes

game_date                     object
home_team                     object
away_team                     object
posteam                       object
defteam                       object
posteam_type                   int32
yardline_100                 float64
quarter_seconds_remaining    float64
half_seconds_remaining       float64
game_seconds_remaining       float64
qtr                            int64
down                         float64
goal_to_go                     int64
ydstogo                        int64
play_type                     object
score_differential           float64
shotgun                        int64
no_huddle                      int64
dtype: object

eagles_df["play_type"].replace({"pass": "1", "run": "0"}, inplace=True)

eagles_df['play_type'] = eagles_df['play_type'].astype(str).astype(int)

In [114]:
final_df = eagles_df[['posteam_type', 'yardline_100', 'quarter_seconds_remaining', 'half_seconds_remaining',
                      'game_seconds_remaining', 'qtr', 'down', 'goal_to_go', 'ydstogo', 'play_type', 'score_differential',
                      'shotgun', 'no_huddle']]

In [115]:
final_df

,posteam_type,yardline_100,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_differential,shotgun,no_huddle
0,1,75.0,654.0,1554.0,3354.0,1,1.0,0,10,pass,-7.0,1,0
1,1,75.0,648.0,1548.0,3348.0,1,2.0,0,10,pass,-7.0,0,0
2,1,64.0,617.0,1517.0,3317.0,1,1.0,0,10,run,-7.0,0,0
3,1,62.0,575.0,1475.0,3275.0,1,2.0,0,8,run,-7.0,1,0
4,1,71.0,539.0,1439.0,3239.0,1,3.0,0,17,pass,-7.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3526,0,11.0,150.0,150.0,150.0,4,2.0,0,7,pass,-1.0,1,0
3527,0,11.0,145.0,145.0,145.0,4,3.0,0,7,pass,-1.0,1,0
3528,0,31.0,129.0,129.0,129.0,4,1.0,0,10,run,5.0,0,0
3529,0,29.0,123.0,123.0,123.0,4,2.0,0,8,run,5.0,0,0


In [131]:
# Split the outcome and predictor variables
target = final_df['play_type']
features = final_df.drop("play_type", axis=1)

In [133]:
features.dtypes

posteam_type                   int32
yardline_100                 float64
quarter_seconds_remaining    float64
half_seconds_remaining       float64
game_seconds_remaining       float64
qtr                            int64
down                         float64
goal_to_go                     int64
ydstogo                        int64
score_differential           float64
shotgun                        int64
no_huddle                      int64
dtype: object

In [94]:
import sweetviz

In [98]:
my_report = sweetviz.compare([training_df, "Train"], [testing_df, "Test"], "play_type")


                                   |                         | [  0%]   00:00  -> (? left)
Summarizing dataframe:             |                         | [  0%]   00:00  -> (? left)
:TARGET::                          |█                    | [  5%]   00:00  -> (00:00 left)
:TARGET::                          |██▏                  | [ 11%]   00:00  -> (00:02 left)
:game_date:                        |██▏                  | [ 11%]   00:00  -> (00:02 left)
:game_date:                        |███▎                 | [ 16%]   00:00  -> (00:04 left)
:home_team:                        |███▎                 | [ 16%]   00:00  -> (00:04 left)
:home_team:                        |████▍                | [ 21%]   00:01  -> (00:04 left)
:away_team:                        |████▍                | [ 21%]   00:01  -> (00:04 left)
:away_team:                        |█████▌               | [ 26%]   00:01  -> (00:04 left)
:posteam:                          |█████▌               | [ 26%]   00:01  -> (00:04 left

Creating Associations graph... DONE!


In [99]:
my_report.show_html("Report.html") # Not providing a filename will default to SWEETVIZ_REPORT.html